In [2]:
import json
import numpy as np
from bottom_up.all import pick_bfs_connected
from bottom_up.all import get_lr
from bottom_up.all import get_dependents
from bottom_up.all import get_f1
from bottom_up.all import featurize_ultra_local
from bottom_up.all import get_local_feats
from bottom_up.all import get_global_feats
from bottom_up.all import featurize_parent_proposal
from bottom_up.all import pick_l2r_connected
from bottom_up.all import featurize_child_proposal
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from code.printers import pretty_print_conl
from bottom_up.all import pick_bfs_connected
from bottom_up.all import len_tree

from bottom_up.all import get_labels_and_features
from bottom_up.all import get_governor

import json
import pickle
import numpy as np
from bottom_up.all import get_lr
from bottom_up.all import get_dependents
from bottom_up.all import get_f1
from bottom_up.all import featurize_ultra_local
from bottom_up.all import featurize_parent_proposal
from bottom_up.all import featurize_child_proposal
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from code.printers import pretty_print_conl
from bottom_up.all import len_tree

from code.treeops import bfs
from bottom_up.all import pick_bfs
from bottom_up.all import get_dependents_and_governors
import random


def train_clf(training_paths = "training.paths", validiation_paths="validation.paths"):

    training_paths = [_ for _ in open(training_paths)]
    validation_paths = [_ for _ in open(validiation_paths)]


    vectorizer = DictVectorizer(sparse=True)

    train_features, train_labels = get_labels_and_features(training_paths)

    X_train = vectorizer.fit_transform(train_features)

    y_train = np.asarray(train_labels)

    val_features, val_labels = get_labels_and_features(validation_paths)

    X_val = vectorizer.transform(val_features)

    y_val = np.asarray(val_labels)

    clf = LogisticRegression(random_state=0,
                             solver='lbfgs',
                             C=.05,
                             multi_class='ovr').fit(X_train, y_train)

    print(clf.score(X_val,y_val))
    print(clf.score(X_train, y_train))
    return clf, vectorizer


def runtime_path_wild_frontier(sentence, pi, clf, vectorizer, verbose=False):
    T = {i for i in sentence["q"]}
    F = set()
    
    # init frontier
    for v in T:
        for i in sentence["tokens"]:
            F.add(i["index"])

    d, heads, c = bfs(g=sentence, hop_s=0) 
    while len(F) > 0:
        vertex = pi(F=F, d=d, T=T, s=sentence)

        if vertex != 0: # bug here?
            feats = get_local_feats(vertex=vertex, sentence=sentence, d=d, current_tree=T)
            feats = get_global_feats(sentence=sentence, feats=feats, vertex=vertex, current_tree=T)

            X = vectorizer.transform([feats])
            pred = clf.predict_proba(X)[0]
            y = clf.predict(X)[0]

            if y == 1:
                T.add(vertex)
                if vertex != 0:
                    for i in get_dependents_and_governors(vertex, sentence, T):
                        if i not in T and i is not None:
                            F.add(i)
                else:
                    for i in get_dependents(sentence, vertex):
                        if i["dependent"] not in T and i is not None:
                            F.add(i)
        F.remove(vertex)

    return T


clf, vectorizer = train_clf()

tot = 0
for pno, paths in enumerate(open("validation.paths", "r")):
    paths=json.loads(paths)
    sentence = paths["sentence"]
    predicted = runtime_path_wild_frontier(sentence, pi=pick_l2r_connected, clf=clf, vectorizer=vectorizer)
    f1s = get_f1(predicted, sentence)
    tot += f1s
print(tot/(pno + 1))

0.9470140978932362
0.9546717310439
0.8576927098089177


In [186]:

### this is just debugging code

confusion = []
from collections import Counter
from collections import defaultdict
all_vals = defaultdict(list)
scr = clf.predict(X_train)
o = 0
h = 0
tot = []
big_list = []
big_list_sentences = []

for paths in training_paths:
    paths=json.loads(paths)
    s = paths["sentence"]
    for p in paths["paths"]:
        current_tree, vertex, decision = p
        if vertex != 0:  
            big_list.append(p)
            big_list_sentences.append(s)
        

for ino, i in enumerate(train_features):
    if 'dep' in i and i["dep"] == "amod":
        o += 1
        if scr[ino] == y_train[ino]:
            h += 1
        else:
            tot.append((scr[ino], y_train[ino], big_list_sentences[ino],train_features[ino], big_list[ino]))
print(h/(len(tot) + h))
print(len(tot))


hits = 0
govs = defaultdict(int)
import string
punct = [_ for _ in string.punctuation]
alls= []
for t in tot:
    pred, was, s, feats, path = t
    current_tree, vertex, decision = path
    current_tree.sort()
    governed = [_["dep"] for _ in s["basicDependencies"] if _["dependent"] == vertex]
    for g in governed:
        govs[g] += 1
    if(pred == 0 and was == 1):
        print(feats,pred,was)
        
        
Counter(alls).most_common()
#print(hits)
#print(len(tot))
#print(sum(govs.values()))

0.7480967917346384
1853
